In [1]:
import requests
from bs4 import BeautifulSoup as bs
import os
import pandas as pd
from pyxlsb import open_workbook
import datetime as dt
import re


#Go to desired directory
#os.chdir("") 

#Find baker hughes website and convert to html
url = "https://rigcount.bakerhughes.com/na-rig-count"
r = requests.get(url)
soup = bs(r.text, "html.parser")

#Find all hyperlinks who's title contains 'Pivot' and pull the first one's address
pivot_file = soup.find_all('a', title = re.compile('pivot'))
link = pivot_file[0]['href']


r = requests.get(link)

#Open a workbook and write the binary content into it
output = open('baker_hughes_rig_count_binary.xlsb', 'wb')
output.write(r.content)
output.close()

df = []

#write the binary workbook to an empty dataframe
with open_workbook('baker_hughes_rig_count_binary.xlsb') as wb:
    with wb.get_sheet("Master Data") as sheet:
        for row in sheet.rows():
            df.append([item.v for item in row])

df = pd.DataFrame(df[1:], columns=df[0])

#Convert Publish date to an excel date format
df['PublishDate'] = pd.TimedeltaIndex(df['PublishDate'], unit= 'd') + dt.datetime(1899, 12, 30)

In [2]:
#Add the most recent rig count date
df['One Week Earlier'] = df['PublishDate'].max() -  pd.to_timedelta(7, unit='d')

In [3]:
#Caclulate Rig count per basin one week earlier
df1 = df[df['One Week Earlier'] == df['PublishDate']].groupby(['Basin'])['RigCount'].agg('sum').reset_index()
df1.rename(columns = {"RigCount": "One Week Earlier Rig Total"},  inplace = True) 
df = df.merge(df1,on='Basin', how='left')

In [4]:
#Calculate Rig count per Basin of current week
df2 = df[df['PublishDate'] == df['PublishDate'].max()].groupby(['Basin'])['RigCount'].agg('sum').reset_index()
df2.rename(columns = {"RigCount": "Newest Rig Total"},  inplace = True) 
df = df.merge(df2,on='Basin', how='left')

In [5]:
#Calculate week over week differential for Basin
df['Weekly Change'] = df['Newest Rig Total'] - df['One Week Earlier Rig Total']

In [6]:
df['Percent Change'] = df['Weekly Change']/df['One Week Earlier Rig Total']

In [7]:
df = df.drop(['One Week Earlier'], axis=1)

In [8]:
#Write to an excel workbook
writer = pd.ExcelWriter('baker_hughes.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Data', index=False)
